# Debut

In [95]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

In [87]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')  # Remplacez par le nom de votre base de données
cursor = conn.cursor()


# 1. Ajouter la colonne `Index_precedent`
try:
    cursor.execute('''
        ALTER TABLE Qte_Consommation ADD COLUMN Index_precedent REAL;
    ''')
    print("Colonne 'Index_precedent' ajoutée avec succès.")
except sqlite3.OperationalError as e:
    print(f"Erreur lors de l'ajout de la colonne : {e}")

# 2. Remplir la colonne `Index_precedent` avec les valeurs appropriées
try:
    cursor.execute('''
        UPDATE Qte_Consommation
        SET Index_precedent = (
            SELECT Index
            FROM Qte_Consommation AS prev
            WHERE prev.N_contrat = Qte_Consommation.N_contrat
              AND prev.mois_consome < Qte_Consommation.mois_consome
            ORDER BY prev.mois_consome DESC
            LIMIT 1
        );
    ''')
    print("Colonne 'Index_precedent' remplie avec succès.")
except sqlite3.OperationalError as e:
    print(f"Erreur lors de la mise à jour de la colonne : {e}")


# 1. Ajouter la colonne `gestion`
try:
    cursor.execute('''
        ALTER TABLE Qte_Consommation ADD COLUMN gestion REAL;
    ''')
    print("Colonne 'gestion' ajoutée avec succès.")
except sqlite3.OperationalError as e:
    print(f"Erreur lors de l'ajout de la colonne : {e}")

# 1. Ajouter la colonne `perte`
try:
    cursor.execute('''
        ALTER TABLE Qte_Consommation ADD COLUMN perte REAL;
    ''')
    print("Colonne 'perte' ajoutée avec succès.")
except sqlite3.OperationalError as e:
    print(f"Erreur lors de l'ajout de la colonne : {e}")

# 3. Valider les modifications et fermer la connexion
conn.commit()
conn.close()

Colonne 'Index_precedent' ajoutée avec succès.
Erreur lors de la mise à jour de la colonne : near "Index": syntax error
Colonne 'gestion' ajoutée avec succès.
Colonne 'perte' ajoutée avec succès.


In [10]:
DELETE FROM Produit_Utiliser;
DELETE FROM Produit_Acheter;
DELETE FROM Qte_Consommation;
DELETE FROM Pay_Consommation;
DELETE FROM ONEP_Payment;
DELETE FROM ONEP_Credit;
DELETE FROM Maintenance_Details;
DELETE FROM Achat;


DELETE FROM Produit;
DELETE FROM Abonnement;
DELETE FROM Maintenance;
DELETE FROM Blocks;
DELETE FROM info_personne

[]


In [206]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()
cursor.execute('''
DELETE FROM Abonnement
''')
conn.commit()
conn.close()

In [15]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()
cursor.execute('''
               ALTER TABLE Pay_Consommation ADD COLUMN Perte REAL DEFAULT 0.0;
               ALTER TABLE Pay_Consommation ADD COLUMN Gestion REAL DEFAULT 0.0;

''')

conn.commit()
conn.close()

In [16]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')  # Remplacez par le nom de votre base de données
cursor = conn.cursor()
cursor.execute('DROP VIEW IF EXISTS Mouvements_Caisse;')
conn.close()

In [13]:
import sqlite3

# Connexion à la base de données SQLite
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Liste des tables à supprimer
tables = [
    "info_personne",
    "Abonnement",
    "Qte_Consommation",
    "Pay_Consommation",
    "Produit_Acheter",
    "Produit",
    "Produit_Utiliser",
    "ONEP_Payment",
    "ONEP_Credit",
    "Maintenance",
    "Maintenance_Details",
    "Blocks",
    "Achat"
]

# Générer et exécuter les commandes DROP TABLE IF EXISTS
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
    print(f"Table '{table}' supprimée.")

# Valider les modifications et fermer la connexion
conn.commit()
conn.close()


Table 'info_personne' supprimée.
Table 'Abonnement' supprimée.
Table 'Qte_Consommation' supprimée.
Table 'Pay_Consommation' supprimée.
Table 'Produit_Acheter' supprimée.
Table 'Produit' supprimée.
Table 'Produit_Utiliser' supprimée.
Table 'ONEP_Payment' supprimée.
Table 'ONEP_Credit' supprimée.
Table 'Maintenance' supprimée.
Table 'Maintenance_Details' supprimée.
Table 'Blocks' supprimée.
Table 'Achat' supprimée.


In [58]:
cursor.execute('''
CREATE VIEW IF NOT EXISTS Mouvements_Caisse AS
SELECT N_contrat AS ID, Date_payement AS Date_Mouvement, 'Adhision' AS Motif, Mnt_paye AS Montant
FROM Abonnement
UNION ALL
SELECT N_contrat AS ID, Date_payement AS Date_Mouvement, 'Consommation' AS Motif, Mnt_paye AS Montant
FROM Pay_Consommation
UNION ALL
SELECT ID_Payment AS ID, Date_Reglement AS Date_Mouvement, 'ONEP' AS Motif, Mnt_paye AS Montant
FROM ONEP_Payment
UNION ALL
SELECT ID_Maintenance AS ID, Date_operation AS Date_Mouvement, 'Maintenance' AS Motif, Mnt_ouvrier AS Montant 
FROM Maintenance
UNION ALL
SELECT ID_Achat AS ID, Date_Achat AS Date_Mouvement, 'Charge Materiel' AS Motif, -Montant_total AS Montant
FROM Produit_Acheter
''')


In [55]:
cursor.execute("DROP VIEW IF EXISTS Mouvements_Caisse")


In [2]:
# Étape 1 : Créer une table temporaire sans clé primaire
cursor.execute('''
CREATE TABLE IF NOT EXISTS Blocks_temp (
    Block_ID INTEGER,
    Index_m3 REAL,
    Date_consome DATE
);
''')

# Étape 2 : Copier les données de l'ancienne table vers la nouvelle table
cursor.execute('''
INSERT INTO Blocks_temp (Block_ID, Index_m3, Date_consome)
SELECT Block_ID, Index_m3, Date_consome FROM Blocks;
''')

# Étape 3 : Supprimer l'ancienne table
cursor.execute('DROP TABLE Blocks;')

# Étape 4 : Renommer la table temporaire
cursor.execute('ALTER TABLE Blocks_temp RENAME TO Blocks;')

# Sauvegarder les modifications
conn.commit()
conn.close()

print("La clé primaire a été supprimée avec succès.")



La clé primaire a été supprimée avec succès.


In [ ]:

cursor.execute('''DELETE FROM ONEP_Payment
WHERE Recu = 4444;''')


In [2]:
cursor.execute('''
INSERT INTO Produit (Nom_Produit) VALUES
    ('Compteur'),
    ('Pompe'),
    ('Filtre'),
    ('Réservoir'),
    ('Tuyau irrigation'),
    ('Capteur de niveau'),
    ('Vanne à bille'),
    ('Distributeur'),
    ('Adoucisseur'),
    ('Kit de traitement'),
    ('Divers');
''')


# Table 

In [14]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Création de la table info_personne
cursor.execute('''
CREATE TABLE IF NOT EXISTS info_personne (
    N_contrat TEXT PRIMARY KEY,
    Cin TEXT,
    N_conpteur_B TEXT,
    N_conpteur_P TEXT,
    Nom TEXT NOT NULL,
    Mnt_due REAL,
    Date_Adhesion DATE,
    Adresse TEXT
)
''')

# Création de la table Abonnement
cursor.execute('''
CREATE TABLE IF NOT EXISTS Abonnement (
    N_contrat TEXT,
    N_recue TEXT,
    Mnt_paye REAL,
    Date_payement DATE,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat)
)
''')

# Création de la table Qte_Consommation
cursor.execute('''
CREATE TABLE IF NOT EXISTS Qte_Consommation (
    N_contrat TEXT,
    Date_consome DATE,
    Quantite REAL,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat)
)
''')

# Création de la table Pay_Consommation
cursor.execute('''
CREATE TABLE IF NOT EXISTS Pay_Consommation (
    N_contrat TEXT,
    N_recue TEXT,
    Mnt_paye REAL,
    Date_paye Date,
    Date_payement DATE,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat))
''')

"""# Table Mouvement_Stock 
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Mouvement_Stock (
    ID_Mouvement INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_Mouvement DATE,
    Type_Mouvement TEXT,  -- Par exemple : "entrée", "sortie", "retour"
    Quantite REAL,  -- Quantité ajoutée ou retirée
    Description TEXT  -- Détails du mouvement (par exemple : "Sortie pour installation")
)
''')"""

# Table Achat 
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit_Acheter (
    ID_Achat INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_Achat DATE,
    Quantite_achetee REAL,
    Prix_unitaire REAL,
    Montant_total REAL AS (Quantite_achetee * Prix_unitaire),
    N_recu TEXT,
    Fournisseur TEXT
)
''')

# Table Produit
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit (
    Nom_Produit TEXT PRIMARY KEY NOT NULL
)
''')

# Table Produit_Utiliser
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit_Utiliser (
    ID_utils INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_utils DATE,
    Quantite_utuli REAL,
    Description TEXT
)
''')

#Table ONEP_Payement
cursor.execute('''CREATE TABLE IF NOT EXISTS ONEP_Payment (
    ID_Payment INTEGER PRIMARY KEY AUTOINCREMENT, 
    Mois_consome TEXT NOT NULL,            
    Mnt_paye REAL NOT NULL,                
    Recu TEXT NOT NULL,                    
    Date_Reglement DATE NOT NULL                  
)''')

# Table ONEP_Charge
cursor.execute('''CREATE TABLE IF NOT EXISTS ONEP_Credit (
    ID_Charge INTEGER PRIMARY KEY AUTOINCREMENT,  
    Mois_consome TEXT NOT NULL,  
    Quantite_Consome REAL,
    Mnt_a_paye REAL NOT NULL,                
    Facture TEXT                             
) ''')

# Table Maintenance
cursor.execute(''' CREATE TABLE IF NOT EXISTS Maintenance (
    ID_Maintenance INTEGER PRIMARY KEY AUTOINCREMENT,
    Date_operation DATE NOT NULL,
    Description TEXT,
    Nom_ouvrier TEXT,
    Mnt_ouvrier REAL
)
''')
               
# Table Maintenance en detail
cursor.execute(''' CREATE TABLE IF NOT EXISTS Maintenance_Details (
    ID_Detail INTEGER PRIMARY KEY AUTOINCREMENT,
    ID_Maintenance INTEGER REFERENCES Maintenance(ID_Maintenance) ON DELETE CASCADE,
    Nom_produit TEXT REFERENCES Produit(Nom_Produit),
    Quantite_produit REAL NOT NULL,
    FOREIGN KEY (ID_Maintenance) REFERENCES Maintenance(ID_Maintenance)
)
''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Blocks (
    Block_ID INTEGER ,
    Index_m3 REAL NOT NULL,
    Date_consome Date
);
       ''')

# Inserion 

In [15]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Données à insérer
data_info_personne = [
    ('01', 'Bk234', '1', 'CP001', 'John Doe', 1500.50, '2024-01-01', 'hamou'),
    ('02', 'BK123', '2', 'CP002', 'Jane Smith', 2000.00, '2024-02-01', 'baala'),
    ('03', 'BK45', '3', 'CP003', 'Ahmed Ali', 1750.75, '2024-03-10', '789 Boulevard Zerktouni, Fès'),
    ('04', 'CIN004', '2', 'CP004', 'Sarah Karim', 1850.00, '2024-03-15', '321 Rue Ibn Battuta, Marrakech'),
    ('05', 'CIN005', '3', 'CP005', 'Youssef Haddad', 1450.25, '2024-04-01', '654 Avenue Hassan II, Tanger'),
    ('06', 'CIN006', '4', 'CP006', 'Fatima Zahra', 2100.00, '2024-04-20', '876 Rue Oqba Ibn Nafia, Agadir'),
    ('07', 'CIN007', '4', 'CP007', 'Mohamed Chraibi', 1950.30, '2024-05-01', '543 Boulevard Moulay Youssef, Oujda'),
    ('08', 'CIN008', '4', 'CP008', 'Hajar El Alaoui', 1800.00, '2024-05-10', '234 Rue Al Mounir, Meknès'),
    ('09', 'CIN009', '1', 'CP009', 'Rachid Benomar', 1650.60, '2024-06-01', '123 Rue Dakhla, Tétouan'),
    ('10', 'CIN010', '3', 'CP010', 'Samira Lahlou', 1750.80, '2024-06-15', '456 Avenue Abdelmoumen, Laâyoune')
]

# Insertion des données
cursor.executemany('''
INSERT INTO info_personne (N_contrat, Cin, N_conpteur_B, N_conpteur_P, Nom, Mnt_due, Date_Adhesion, Adresse)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', data_info_personne)

# Validation des modifications et fermeture de la connexion
conn.commit()
conn.close()

print("Données insérées avec succès.")


Données insérées avec succès.


In [16]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Données à insérer dans la table Abonnement
data_abonnement = [
    ('01', '1', 500.00, '2024-01-10'),
    ('01', '2', 500.00, '2024-02-10'),
    ('01', '3', 500.00, '2024-03-10'),
    ('01', '4', 0.50, '2024-04-01'),
    ('02', '5', 1000.00, '2024-02-15'),
    ('02', '6', 1000.00, '2024-03-15'),
    ('03', '7', 500.00, '2024-03-15'),
    ('03', '8', 500.00, '2024-04-05'),
    ('04', '9', 800.00, '2024-03-20'),
    ('04', '10', 800.00, '2024-04-05'),
    ('05', '11', 800.00, '2024-04-10'),
    ('06', '12', 500.00, '2024-04-25'),
    ('06', '13', 500.00, '2024-05-05'),
    ('07', '14', 1000.00, '2024-05-15'),
    ('08', '15', 1800.00, '2024-05-25'),
    ('09', '16', 1650.60, '2024-06-10'),
    ('10', '17', 600.00, '2024-06-20'),
    ('10', '18', 1150.80, '2024-07-01')
]

# Insertion des données dans la table Abonnement
cursor.executemany('''
INSERT INTO Abonnement (N_contrat, N_recue, Mnt_paye, Date_payement)
VALUES (?, ?, ?, ?)
''', data_abonnement)

# Validation des modifications et fermeture de la connexion
conn.commit()
conn.close()

print("Données insérées avec succès dans la table Abonnement.")


Données insérées avec succès dans la table Abonnement.


In [8]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Données de consommation à insérer
data_consommation = [
    ('01', 2, '2024-06-01'),
    ('01', 4, '2024-07-01'),
    ('01', 5, '2024-08-01'),
    ('01', 6, '2024-09-01'),
    ('01', 7, '2024-10-01'),
    ('01', 10, '2024-11-01'),
    ('02', 3, '2024-06-01'),
    ('02', 9, '2024-07-01'),
    ('02', 14, '2024-08-01'),
    ('02', 18, '2024-09-01'),
    ('02', 19, '2024-10-01'),
    ('02', 22, '2024-11-01'),
    ('03', 1, '2024-06-01'),
    ('03', 7, '2024-07-01'),
    ('03', 10, '2024-08-01'),
    ('03', 11, '2024-09-01'),
    ('03', 15, '2024-10-01'),
    ('03', 20, '2024-11-01'),
    ('04', 10, '2024-06-01'),
    ('04', 13, '2024-07-01'),
    ('04', 19, '2024-08-01'),
    ('04', 22, '2024-09-01'),
    ('04', 26, '2024-10-01'),
    ('04', 28, '2024-11-01'),
    ('05', 1, '2024-06-01'),
    ('05', 7, '2024-07-01'),
    ('05', 9, '2024-08-01'),
    ('05', 17, '2024-09-01'),
    ('05', 20, '2024-10-01'),
    ('05', 28, '2024-11-01')
]

# Insertion des données dans la table Consommation
cursor.executemany('''
INSERT INTO Qte_Consommation (N_contrat, Quantite, Date_consome)
VALUES (?, ?, ?)
''', data_consommation)

# Validation des modifications et fermeture de la connexion
conn.commit()
conn.close()


In [11]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Données à insérer
data_pay_consommation = [
    ('01', '1', 14.00, '2024-06-15', '2024-06-16'),
    ('01', '6', 14.00, '2024-07-01', '2024-07-02'),
    ('01', '5', 7.00, '2024-08-15', '2024-08-16'),
    ('01', '6', 7.00, '2024-09-01', '2024-09-02'),
    ('01', '2', 7.00, '2024-10-15', '2024-10-16'),

    ('02', '2', 21.00, '2024-06-15', '2024-06-16'),
    ('02', '7', 42.00, '2024-07-01', '2024-07-02'),
    ('02', '11', 35.00, '2024-08-15', '2024-08-16'),
    ('02', '15', 28.00, '2024-09-01', '2024-09-02'),
    
    ('03', '9', 7.00, '2024-06-15', '2024-06-16'),
    ('03', '16', 42.00, '2024-07-01', '2024-07-02'),
    ('03', '15', 21.00, '2024-08-15', '2024-08-16'),
    ('03', '16', 7.00, '2024-09-01', '2024-09-02'),
    ('03', '12', 28.00, '2024-10-15', '2024-10-16'),
    ('03', '10', 35.00, '2024-11-15', '2024-11-16'),

    ('04', '3', 21.00, '2024-09-01', '2024-10-02'),
    ('04', '7', 70.00, '2024-06-15', '2024-08-16'),
    ('04', '12', 14.00, '2024-11-01', '2024-11-02'),

    ('05', '12', 7.00, '2024-06-01', '2024-06-02'),
    
]

# Insertion des données
cursor.executemany('''
INSERT INTO Pay_Consommation (N_contrat, N_recue, Mnt_paye, Date_paye, Date_payement)
VALUES (?, ?, ?, ?, ?)
''', data_pay_consommation)

# Validation des changements
conn.commit()

# Fermeture de la connexion
conn.close()

print("Les données ont été insérées dans la table Pay_Consommation avec succès.")


Les données ont été insérées dans la table Pay_Consommation avec succès.


# XXX

In [5]:
cursor.execute("DROP VIEW IF EXISTS Mouvements_Caisse")

# Valider et fermer la connexion
conn.commit()
conn.close()

In [ ]:
import sqlite3

# Connexion à la base de données (remplacez par votre connexion de base de données)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Exécuter la requête SQL
query = '''
SELECT 
    q.N_contrat,
    strftime('%Y-%m', q.Date_consome) AS Mois_Consome,
    q.Date_consome,
    q.Quantite AS Index_m3,
    COALESCE(q.Quantite - LAG(q.Quantite) OVER (PARTITION BY q.N_contrat ORDER BY q.Date_consome), q.Quantite) AS Qte_Consomme_m3,
    (COALESCE(q.Quantite - LAG(q.Quantite) OVER (PARTITION BY q.N_contrat ORDER BY q.Date_consome), q.Quantite) * 7) AS Montant_dh
FROM 
    Qte_Consommation q;

'''

cursor.execute(query)

# Récupérer les résultats
results = cursor.fetchall()

# Afficher les résultats
for row in results:
    print(row)

# Fermer la connexion
conn.close()


('000', '2024-12', '2024-12-01', 90.0, 90.0)
('100', '2024-11', '2024-11-01', 0.0, 0.0)
('100', '2024-11', '2024-11-01', 0.0, 0.0)
('11', '2024-08', '2024-08-01', 60.0, 60.0)
('111', '2024-10', '2024-10-01', 70.0, 70.0)
('111', '2024-11', '2024-11-01', 77.0, 7.0)
('111', '2024-11', '2024-11-13', 129.0, 52.0)
('111', '2024-12', '2024-12-01', 90.0, -39.0)
('123', '2024-11', '2024-11-01', 301.0, 301.0)
('123', '2024-12', '2024-12-01', 12345.0, 12044.0)
('124', '2024-11', '2024-11-01', 56.0, 56.0)
('222', '2024-11', '2024-11-07', 145.0, 145.0)
('222', '2024-12', '2024-12-03', 160.0, 15.0)
('222', '2025-01', '2025-01-02', 179.0, 19.0)
('333', '2024-10', '2024-10-01', 50.0, 50.0)
('333', '2024-11', '2024-11-02', 56.0, 6.0)
('55', '2024-09', '2024-09-01', 5.0, 5.0)
('55', '2024-10', '2024-10-01', 10.0, 5.0)
('55', '2024-11', '2024-11-01', 15.0, 5.0)
('78', '2024-08', '2024-08-01', 12.0, 12.0)
('78', '2024-09', '2024-09-02', 18.0, 6.0)
('78', '2024-10', '2024-10-03', 25.0, 7.0)
('78', '2024-11

In [13]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')

# Création d'un curseur
cursor = conn.cursor()

# Exécution de la requête pour lister les tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Récupération et affichage des noms de tables
tables = cursor.fetchall()
print("Tables dans la base de données Gestion_eau :")
for table in tables:
    print(table[0])

# Fermeture de la connexion
conn.close()


Tables dans la base de données Gestion_eau :
info_personne
Abonnement
Qte_Consommation
Pay_Consommation


In [85]:
# Enregistrer les modifications et fermer la connexion
conn.commit()
conn.close()

# Insertion des donnees reelles

## Info personnelle

In [1]:
import pandas as pd 

In [53]:
fichier_excel= "/Users/mac/Desktop/Gestion-Eau-Association-Nasr/Donnee Brute/MB suivi de compte.xlsx"
noms_colonnes = ["N_Contrat", "Nom_complet", "Mnt_due", "Mnt_paye"]
df = pd.read_excel(fichier_excel, sheet_name="Sheet1",names=noms_colonnes)
df

,N_Contrat,Nom_complet,Mnt_due,Mnt_paye
0,1,عبد الحق الحمزوي,3600,3600
1,2,صالح حمدي,3600,3600
2,3,العربي الزعاري,3600,3600
3,4,أحمد لعناني,3600,1000
4,5,عبد الكريم النخالي,3600,3600
...,...,...,...,...
149,150,نبيل حمدي,3600,3600
150,151,مصطفى كعموش,4000,1500
151,152,نعيمة مداح,4000,4000
152,153,مباركة حمدان,4000,4000


In [54]:
import pandas as pd
import sqlite3  # Remplacez par le module approprié si vous utilisez une autre base de données

# 1. Importer les données depuis le fichier Excel
fichier_excel= "/Users/mac/Desktop/Gestion-Eau-Association-Nasr/Donnee Brute/MB suivi de compte.xlsx"
feuille = "Sheet1"
df = pd.read_excel(fichier_excel, sheet_name=feuille)

# 2. Sélectionner les colonnes nécessaires
colonnes_a_inserer = ["N_Contrat", "Nom_complet", "Mnt_due"]
df_selectionne = df[colonnes_a_inserer]

# 3. Se connecter à la base de données
# Remplacez 'Gestion_eau.db' par le nom de votre base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# 4. Insérer les données dans la table info_personne
for _, row in df_selectionne.iterrows():
    cursor.execute('''
        INSERT INTO info_personne (N_contrat, Nom, Mnt_due)
        VALUES (?, ?, ?)
    ''', (row["N_Contrat"], row["Nom_complet"], row["Mnt_due"]))

# 5. Valider les changements et fermer la connexion
conn.commit()
conn.close()

print("Données insérées avec succès dans la table info_personne.")

Données insérées avec succès dans la table info_personne.


## Consommation

In [88]:
fichier_excel= "/Users/mac/Desktop/Gestion-Eau-Association-Nasr/Donnee Brute/MB suivi de compte.xlsx"
#noms_colonnes = ["N_Contrat", "Nom_complet", "Index_initial", "Index_NV"]
df = pd.read_excel(fichier_excel, sheet_name="Sheet6")
df

,N_contrat,Nom,Index_initial,Jan,Fev,Mars,Avr,Mai,Juin,Juil,Aout,Sep,Oct
0,6,حسن البرني,608.0,608.0,608.0,611.0,617.0,628.0,636.0,649.0,660.0,671.0,677.0
1,73,سعيد البرني,145.0,145.0,145.0,1.0,6.0,11.0,16.0,21.0,28.0,35.0,39.0
2,55,البرني لحسن (مليكة),318.0,318.0,318.0,319.0,320.0,323.0,330.0,338.0,344.0,354.0,364.0
3,30,المصطفى البرني,579.0,580.0,580.0,582.0,590.0,600.0,607.0,613.0,621.0,628.0,633.0
4,69,رشيد النافع,291.0,292.0,292.0,294.0,298.0,302.0,307.0,311.0,316.0,322.0,327.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,111,محمد الزعاري,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,114,محمد وجيه,81.0,81.0,81.0,81.0,81.0,81.0,81.0,81.0,81.0,81.0,81.0
152,133,محمد محمد أوساني,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
153,145,محمد بن احمد نافع,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df.isna().sum()


N_contrat         0
Nom_complet       0
Index_initial    10
Jan              12
Fev               9
Mars             12
Avr               8
Mai               8
Juin              8
Juil              8
Aout              7
Sep               7
Oct               7
dtype: int64

In [89]:
df.fillna(0, inplace=True)
display(df.isna().sum())

N_contrat        0
Nom              0
Index_initial    0
Jan              0
Fev              0
Mars             0
Avr              0
Mai              0
Juin             0
Juil             0
Aout             0
Sep              0
Oct              0
dtype: int64

In [ ]:
for i in df['N_contrat']:
    display(df[df['N_contrat']==i])

In [90]:
import pandas as pd
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')  # Remplacez par le nom de votre base de données
cursor = conn.cursor()

# Mois et années correspondants
mois = {
    "Dec": ("2023-12-01", "Index_initial"),  # Décembre 2023
    "Jan": ("2024-01-01", "Jan"),            # Janvier 2024
    "Fev": ("2024-02-01", "Fev"),            # Février 2024
    "Mars": ("2024-03-01", "Mars"),          # Mars 2024
    "Avr": ("2024-04-01", "Avr"),            # Avril 2024
    "Mai": ("2024-05-01", "Mai"),            # Mai 2024
    "Juin": ("2024-06-01", "Juin"),          # Juin 2024
    "Juil": ("2024-07-01", "Juil"),          # Juillet 2024
    "Aout": ("2024-08-01", "Aout"),          # Août 2024
    "Sep": ("2024-09-01", "Sep"),            # Septembre 2024
    "Oct": ("2024-10-01", "Oct")             # Octobre 2024
}

# Insérer les données dans la table Qte_Consommation
for mois_nom, (date, colonne) in mois.items():
    for _, row in df.iterrows():
        # Calculer l'index précédent
        if mois_nom == "Dec":
            index_precedent = None  # Pas de mois précédent pour Décembre
        else:
            # Récupérer l'index du mois précédent
            mois_precedent = list(mois.keys())[list(mois.keys()).index(mois_nom) - 1]
            index_precedent = row[mois[mois_precedent][1]]  # Accéder à la valeur du mois précédent

        # Définir la valeur de `perte`
        if mois_nom in ["Dec", "Jan", "Fev", "Mars", "Avr", "Mai", "Juin"]:
            perte = 0.0
        else:
            perte = 5.0

        # Définir la valeur de `gestion`
        if mois_nom in ["Dec", "Jan", "Fev"]:
            gestion = 0.0
        else:
            gestion = 10.0

        # Insérer les données
        cursor.execute('''
            INSERT INTO Qte_Consommation (N_Contrat, date_consome, Quantite, gestion, perte, index_precedent)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (row["N_contrat"], date, row[colonne], gestion, perte, index_precedent))

# Valider les modifications et fermer la connexion
conn.commit()
conn.close()

print("Données insérées avec succès dans la table Qte_Consommation.")

Données insérées avec succès dans la table Qte_Consommation.


In [ ]:
'''
probleme dans les indexs
N_Contrat: 
3mars,73mars,147mars,72juin,127juin,10mars,102Dec
102mars,106mars,33mai,11mars,137mars,53Juin
'''

import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = -4
    WHERE N_contrat = 73 AND Quantite = 1
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = -2
    WHERE N_contrat = 147 AND Quantite = 1
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = -7
    WHERE N_contrat = 72 AND Quantite = 6
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = -1
    WHERE N_contrat = 127 AND Quantite = 7
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = 1
    WHERE N_contrat = 102 AND Quantite = 1
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = -6
    WHERE N_contrat = 106 AND Quantite = 1
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = 0
    WHERE N_contrat = 33 AND Quantite = 2
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = 6
    WHERE N_contrat = 11 AND Quantite = 7
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = -8
    WHERE N_contrat = 137 AND Quantite = 1
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = -11
    WHERE N_contrat = 10 AND Quantite = 4
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = 1
    WHERE N_contrat = 53 AND Quantite = 12
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = 286
    WHERE N_contrat = 102 AND Date_consome like '2024-01-01'
''')
cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = 308
    WHERE N_contrat = 3 AND Date_consome like '2024-03-01'
''')

# 3. Valider les modifications et fermer la connexion
conn.commit()
conn.close()

In [26]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

cursor.execute('''
    UPDATE Qte_Consommation
    SET Index_precedent = 308
    WHERE N_contrat = 3 AND Date_consome like '2024-03-01'
''')

# 3. Valider les modifications et fermer la connexion
conn.commit()
conn.close()

## Payement Consommation

In [348]:
import pandas as pd
fichier_excel= "/Users/mac/Desktop/Gestion-Eau-Association-Nasr/Donnee Brute/Mnt_consommation_finale_2024.xlsx"
df1 = pd.read_excel(fichier_excel, sheet_name="Sheet1_rehal")

In [349]:
# Afficher les premières lignes
df1

,الاسم الكامل,N_contrat,mois_consome,Crédit,Consom Mensuelle,perte,FG,Total,Total SANS FG,Unnamed: 9,...,Date encaissements,Reçu N°,Montants Brut,perte,FG.1,Montants_Net,Total_recettes,Soldes,Unnamed: 19,Unnamed: 20
0,عبد الحق الحمزوي,1,2024-01-01,NaN,189.0,NaN,10.0,199.0,189.0,NaN,...,2024-04-12 00:00:00,162.0,416,NaN,20.0,396.0,416.0,-217.0,NaN,NaN
1,عبد الحق الحمزوي,1,2024-03-01,NaN,NaN,NaN,NaN,0.0,0.0,NaN,...,2024-05-25 00:00:00,117.0,94,NaN,10.0,84.0,94.0,-94.0,NaN,NaN
2,عبد الحق الحمزوي,1,2024-04-01,MANQUE RECU,70.0,NaN,10.0,80.0,70.0,NaN,...,2024-06-20 00:00:00,106.0,31,NaN,10.0,21.0,31.0,49.0,NaN,NaN
3,عبد الحق الحمزوي,1,2024-05-01,NaN,63.0,NaN,10.0,73.0,63.0,NaN,...,2024-06-20 00:00:00,105.0,73,NaN,10.0,63.0,73.0,0.0,NaN,NaN
4,عبد الحق الحمزوي,1,2024-06-01,NaN,126.0,NaN,10.0,136.0,126.0,NaN,...,2024-07-24 00:00:00,109.0,136,NaN,10.0,126.0,136.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,NaN,Total général,NaT,27098,63450.0,4435.0,14540.0,109523.0,91233.0,NaN,...,NaN,NaN,101065,4485.0,14420.0,82161.0,84062.0,8630.0,NaN,NaN
1994,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [350]:
print(df1.columns)

Index(['الاسم الكامل', 'N_contrat', 'mois_consome', 'Crédit',
       'Consom Mensuelle', ' perte', 'FG', 'Total', 'Total SANS FG',
       'Unnamed: 9', 'Caissier', 'Date encaissements', 'Reçu N°',
       'Montants Brut', 'perte', 'FG.1', 'Montants_Net', 'Total_recettes',
       'Soldes', 'Unnamed: 19', 'Unnamed: 20'],
      dtype='object')


In [351]:
df1 = df1.drop(columns=['Crédit',' perte', 'FG', 'Total SANS FG', 'Total',
       'Unnamed: 9', 'Caissier','Montants Brut','Montants_Net', 'Unnamed: 19', 'Unnamed: 20'])
print(df1.columns)

Index(['الاسم الكامل', 'N_contrat', 'mois_consome', 'Consom Mensuelle',
       'Date encaissements', 'Reçu N°', 'perte', 'FG.1', 'Total_recettes',
       'Soldes'],
      dtype='object')


In [352]:
df1.head(20)

,الاسم الكامل,N_contrat,mois_consome,Consom Mensuelle,Date encaissements,Reçu N°,perte,FG.1,Total_recettes,Soldes
0,عبد الحق الحمزوي,1,2024-01-01,189.0,2024-04-12 00:00:00,162.0,NaN,20.0,416.0,-217.0
1,عبد الحق الحمزوي,1,2024-03-01,NaN,2024-05-25 00:00:00,117.0,NaN,10.0,94.0,-94.0
2,عبد الحق الحمزوي,1,2024-04-01,70.0,2024-06-20 00:00:00,106.0,NaN,10.0,31.0,49.0
3,عبد الحق الحمزوي,1,2024-05-01,63.0,2024-06-20 00:00:00,105.0,NaN,10.0,73.0,0.0
4,عبد الحق الحمزوي,1,2024-06-01,126.0,2024-07-24 00:00:00,109.0,NaN,10.0,136.0,0.0
5,عبد الحق الحمزوي,1,2024-07-01,147.0,2024-08-20 00:00:00,81.0,5.0,10.0,162.0,0.0
6,عبد الحق الحمزوي,1,2024-08-01,84.0,2024-09-17 00:00:00,39.0,5.0,10.0,99.0,0.0
7,عبد الحق الحمزوي,1,2024-09-01,154.0,2024-10-18 00:00:00,52.0,5.0,10.0,169.0,0.0
8,عبد الحق الحمزوي,1,2024-10-01,588.0,NaN,NaN,5.0,10.0,603.0,0.0
9,عبد الحق الحمزوي,1,2024-11-01,371.0,2024-12-18 00:00:00,81.0,5.0,10.0,386.0,0.0


In [353]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997 entries, 0 to 1996
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   الاسم الكامل        1666 non-null   object        
 1   N_contrat           1990 non-null   object        
 2   mois_consome        1818 non-null   datetime64[ns]
 3   Consom Mensuelle    1692 non-null   float64       
 4   Date encaissements  1535 non-null   object        
 5   Reçu N°             1464 non-null   float64       
 6   perte               1056 non-null   float64       
 7   FG.1                1634 non-null   float64       
 8   Total_recettes      1990 non-null   float64       
 9   Soldes              1989 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 156.1+ KB


In [354]:
# gerer les valeur null
df1['N_contrat'] = df1['N_contrat'].astype(str)
df1['Reçu N°'] = df1['Reçu N°'].astype(str)
df1['FG.1'] = df1['FG.1'].fillna(0)
df1['perte'] = df1['perte'].fillna(0)
df1['mois_consome'] = df1['mois_consome'].dt.date

# supprimer les ligne de 001 pour les verifier apres
df1 = df1.iloc[12:,:]

#Supprimer les ligne ou il ya total
df1 = df1[~df1['N_contrat'].str.startswith('Total')]


In [355]:
df1

,الاسم الكامل,N_contrat,mois_consome,Consom Mensuelle,Date encaissements,Reçu N°,perte,FG.1,Total_recettes,Soldes
12,NaN,2,2024-01-01,28.0,2024-04-08 00:00:00,96.0,0.0,0.0,28.0,0.0
13,صالح حمدي,2,2024-03-01,14.0,2024-04-08 00:00:00,96.0,0.0,10.0,24.0,0.0
14,صالح حمدي,2,2024-04-01,119.0,2024-05-08 00:00:00,46.0,0.0,10.0,129.0,0.0
15,صالح حمدي,2,2024-05-01,133.0,2024-06-09 00:00:00,60.0,0.0,10.0,143.0,0.0
16,صالح حمدي,2,2024-06-01,168.0,2024-07-19 00:00:00,95.0,0.0,10.0,178.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1991,NaN,nan,NaT,NaN,NaN,nan,0.0,0.0,-16831.0,NaN
1992,NaN,nan,NaT,NaN,NaN,nan,0.0,0.0,NaN,NaN
1994,NaN,nan,NaT,NaN,NaN,nan,0.0,0.0,NaN,NaN
1995,NaN,nan,NaT,NaN,NaN,nan,0.0,0.0,NaN,NaN


In [356]:
a=(341,342,1690,1581)
for i in a:
    print(df1.loc[i, 'Date encaissements']) 


02/01/1025
02/01/1025
1111/10/2024
1934-08-30 00:00:00


In [357]:
df1.loc[341, 'Date encaissements'] = '2025-01-02'
df1.loc[342, 'Date encaissements'] = '2025-01-02'
df1.loc[1690, 'Date encaissements'] = '2024-10-11'
df1.loc[1581, 'Date encaissements'] = '2024-08-30'


In [358]:
a=[]
for i in df1['Date encaissements'].astype(str):
    if (len(i) < 4) & (i != 'nan'):
        a.append(i)

print(a)

df1[df1['Date encaissements'].isin(['NR', ','])]


['NR', 'NR', ',']


,الاسم الكامل,N_contrat,mois_consome,Consom Mensuelle,Date encaissements,Reçu N°,perte,FG.1,Total_recettes,Soldes
1569,مصطفى بن لكبير أزعاري,123,2024-08-01,35.0,NR,nan,5.0,10.0,0.0,50.0
1635,مصطفى دعنون,128,2024-08-01,21.0,NR,nan,5.0,10.0,0.0,36.0
1847,احمد بنبراهيم ازعاري,144,2024-10-01,56.0,",",74.0,5.0,10.0,71.0,0.0


In [359]:
df1.loc[1847,'Date encaissements'] = '2024-11-4'
import numpy as np
df1.loc[1569, 'Date encaissements'] = np.nan
df1.loc[1635, 'Date encaissements'] = np.nan

In [360]:
df1['Date encaissements'] = pd.to_datetime(df1['Date encaissements'], errors='coerce')
df1['Date encaissements'] = df1['Date encaissements'].dt.date

In [361]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1829 entries, 12 to 1996
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   الاسم الكامل        1500 non-null   object 
 1   N_contrat           1829 non-null   object 
 2   mois_consome        1807 non-null   object 
 3   Consom Mensuelle    1525 non-null   float64
 4   Date encaissements  1523 non-null   object 
 5   Reçu N°             1829 non-null   object 
 6   perte               1829 non-null   float64
 7   FG.1                1829 non-null   float64
 8   Total_recettes      1822 non-null   float64
 9   Soldes              1821 non-null   float64
dtypes: float64(5), object(5)
memory usage: 221.7+ KB


In [362]:
result = df1[df1['Reçu N°'].apply(lambda x: len(str(x)) < 4)]
result

,الاسم الكامل,N_contrat,mois_consome,Consom Mensuelle,Date encaissements,Reçu N°,perte,FG.1,Total_recettes,Soldes
24,NaN,3,2024-01-01,0.0,NaT,nan,0.0,0.0,0.0,0.0
36,NaN,4,2024-01-01,NaN,NaT,nan,0.0,0.0,0.0,0.0
37,أحمد لعناني,4,2024-03-01,NaN,NaT,nan,0.0,0.0,0.0,0.0
38,أحمد لعناني,4,2024-04-01,NaN,NaT,nan,0.0,0.0,0.0,0.0
39,أحمد لعناني,4,2024-05-01,NaN,NaT,nan,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1991,NaN,nan,NaT,NaN,NaT,nan,0.0,0.0,-16831.0,NaN
1992,NaN,nan,NaT,NaN,NaT,nan,0.0,0.0,NaN,NaN
1994,NaN,nan,NaT,NaN,NaT,nan,0.0,0.0,NaN,NaN
1995,NaN,nan,NaT,NaN,NaT,nan,0.0,0.0,NaN,NaN


In [363]:
df1['Reçu N°'] = df1['Reçu N°'].apply(lambda x: None if len(str(x)) < 4 else x)

In [364]:
df2 = df1[df1['Reçu N°'].isna() & df1['Date encaissements'].isna() & df1['Total_recettes']== 0]
df2[['Reçu N°', 'Date encaissements','Total_recettes']]

,Reçu N°,Date encaissements,Total_recettes
12,96.0,2024-04-08,28.0
13,96.0,2024-04-08,24.0
14,46.0,2024-05-08,129.0
15,60.0,2024-06-09,143.0
16,95.0,2024-07-19,178.0
...,...,...,...
1990,None,None,NaN
1992,None,None,NaN
1994,None,None,NaN
1995,None,None,NaN


In [365]:
# Supprimer les lignes où 'Total_recettes' est nul
df1 = df1.dropna(subset=['Total_recettes'])

# Supprimer les lignes où 'Total_recettes' est égal à 0
df1 = df1[df1['Total_recettes'] != 0]

In [366]:
df1['Reçu N°'] = df1['Reçu N°'].apply(lambda x: str(int(float(x))) if not pd.isna(x) else 0)

In [367]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 12 to 1991
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   الاسم الكامل        1255 non-null   object 
 1   N_contrat           1475 non-null   object 
 2   mois_consome        1469 non-null   object 
 3   Consom Mensuelle    1381 non-null   float64
 4   Date encaissements  1455 non-null   object 
 5   Reçu N°             1475 non-null   object 
 6   perte               1475 non-null   float64
 7   FG.1                1475 non-null   float64
 8   Total_recettes      1475 non-null   float64
 9   Soldes              1474 non-null   float64
dtypes: float64(5), object(5)
memory usage: 126.8+ KB


In [368]:
df1[df1['N_contrat'] == '14']

,الاسم الكامل,N_contrat,mois_consome,Consom Mensuelle,Date encaissements,Reçu N°,perte,FG.1,Total_recettes,Soldes
164,NaN,14,2024-01-01,28.0,2024-04-14,170,0.0,0.0,28.0,0.0
165,الجيلالي النخالي,14,2024-03-01,21.0,2024-04-14,170,0.0,10.0,31.0,0.0
166,الجيلالي النخالي,14,2024-04-01,42.0,2024-05-08,48,0.0,10.0,52.0,0.0
167,الجيلالي النخالي,14,2024-05-01,63.0,2024-06-09,57,0.0,10.0,73.0,0.0
168,الجيلالي النخالي,14,2024-06-01,63.0,2024-07-16,80,0.0,10.0,73.0,0.0
169,الجيلالي النخالي,14,2024-07-01,147.0,2024-08-06,15,5.0,10.0,162.0,0.0
170,الجيلالي النخالي,14,2024-08-01,196.0,2024-09-14,15,5.0,10.0,162.0,49.0
171,الجيلالي النخالي,14,NaT,NaN,NaT,0,0.0,0.0,49.0,-49.0
172,الجيلالي النخالي,14,2024-09-01,133.0,2024-10-11,17,5.0,10.0,148.0,0.0
173,الجيلالي النخالي,14,2024-10-01,49.0,2024-11-15,37,5.0,10.0,64.0,0.0


In [369]:
# des mois de consommation sont null mais on les remplace par la date precedente car cest un payment en 2 partie
# Remplacer les valeurs nulles dans 'Mois_consome' par la valeur précédente
df1['mois_consome'] = df1['mois_consome'].fillna(method='ffill')
df1[df1['N_contrat'] == '14']

,الاسم الكامل,N_contrat,mois_consome,Consom Mensuelle,Date encaissements,Reçu N°,perte,FG.1,Total_recettes,Soldes
164,NaN,14,2024-01-01,28.0,2024-04-14,170,0.0,0.0,28.0,0.0
165,الجيلالي النخالي,14,2024-03-01,21.0,2024-04-14,170,0.0,10.0,31.0,0.0
166,الجيلالي النخالي,14,2024-04-01,42.0,2024-05-08,48,0.0,10.0,52.0,0.0
167,الجيلالي النخالي,14,2024-05-01,63.0,2024-06-09,57,0.0,10.0,73.0,0.0
168,الجيلالي النخالي,14,2024-06-01,63.0,2024-07-16,80,0.0,10.0,73.0,0.0
169,الجيلالي النخالي,14,2024-07-01,147.0,2024-08-06,15,5.0,10.0,162.0,0.0
170,الجيلالي النخالي,14,2024-08-01,196.0,2024-09-14,15,5.0,10.0,162.0,49.0
171,الجيلالي النخالي,14,2024-08-01,NaN,NaT,0,0.0,0.0,49.0,-49.0
172,الجيلالي النخالي,14,2024-09-01,133.0,2024-10-11,17,5.0,10.0,148.0,0.0
173,الجيلالي النخالي,14,2024-10-01,49.0,2024-11-15,37,5.0,10.0,64.0,0.0


In [373]:
df1.isna().sum()

الاسم الكامل          220
N_contrat               0
mois_consome            0
Consom Mensuelle       94
Date encaissements     20
Reçu N°                 0
perte                   0
FG.1                    0
Total_recettes          0
Soldes                  1
dtype: int64

In [ ]:
# Je verifie si il ya des valeur null dasn les colonne au dessous

In [374]:
df1[['N_contrat', 'mois_consome', 'Date encaissements', 'Reçu N°', 'Total_recettes']]

,N_contrat,mois_consome,Date encaissements,Reçu N°,Total_recettes
12,2,2024-01-01,2024-04-08,96,28.0
13,2,2024-03-01,2024-04-08,96,24.0
14,2,2024-04-01,2024-05-08,46,129.0
15,2,2024-05-01,2024-06-09,60,143.0
16,2,2024-06-01,2024-07-19,95,178.0
...,...,...,...,...,...
1983,155,2024-12-01,2025-01-12,48,36.0
1985,156,2024-10-01,2024-11-19,68,10.0
1986,156,2024-11-01,2024-12-11,32,43.0
1987,156,2024-12-01,2025-01-20,83,78.0


In [393]:
df1.drop(df1[df1['Total_recettes'] == -16831.0].index, inplace=True)


In [395]:
from datetime import datetime

# Convertir 'mois_consome' et 'Date encaissements' en format date
df1['mois_consome'] = pd.to_datetime(df1['mois_consome'], errors='coerce').dt.date
df1['Date encaissements'] = pd.to_datetime(df1['Date encaissements'], errors='coerce').dt.date


In [403]:
import sqlite3

# Connexion à la base de données SQLite
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Vider la table Pay_Consommation
cursor.execute('DELETE FROM Pay_Consommation')

# Valider les changements
conn.commit()

# Fermer la connexion
conn.close()

In [404]:
import sqlite3
import pandas as pd

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Parcourir chaque ligne du DataFrame
for index, row in df1.iterrows():
    try:
        # Remplacer NaT par None pour les dates
        date_paye = row['mois_consome'] if pd.notna(row['mois_consome']) else None
        date_payement = row['Date encaissements'] if pd.notna(row['Date encaissements']) else None

        # Insérer les données dans la table
        cursor.execute("""
            INSERT INTO Pay_Consommation (N_contrat, N_recue, Mnt_paye, Date_paye, Date_payement, Gestion, Perte)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (row['N_contrat'], row['Reçu N°'], row['Total_recettes'], date_paye, date_payement, row['FG.1'], row['perte']))
    
    except Exception as e:
        print(f"Erreur lors de l'insertion de la ligne {index}: {e}")
        print(f"Valeur problématique: {row.to_dict()}")
        conn.rollback()
        break  
    else:
        conn.commit()

# Fermer la connexion à la base de données
conn.close()

In [381]:
df1.iloc[98]

الاسم الكامل          بوشعيب نعناع
N_contrat                       12
mois_consome            2024-07-01
Consom Mensuelle             147.0
Date encaissements      2024-10-28
Reçu N°                         89
perte                          5.0
FG.1                          10.0
Total_recettes               162.0
Soldes                         0.0
Name: 145, dtype: object

In [382]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 12 to 1991
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   الاسم الكامل        1255 non-null   object 
 1   N_contrat           1475 non-null   object 
 2   mois_consome        1475 non-null   object 
 3   Consom Mensuelle    1381 non-null   float64
 4   Date encaissements  1455 non-null   object 
 5   Reçu N°             1475 non-null   object 
 6   perte               1475 non-null   float64
 7   FG.1                1475 non-null   float64
 8   Total_recettes      1475 non-null   float64
 9   Soldes              1474 non-null   float64
dtypes: float64(5), object(5)
memory usage: 126.8+ KB


In [146]:
import pandas as pd
import sqlite3

# Connexion à la base de données SQLite
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Dictionnaire pour mapper les mois aux colonnes correspondantes dans df2
mois_mapping = {
    'mars': (' cons_mars', 'N_recue_mars', 'Date_mars'),
    'avril': (' cons_avril', 'N_recue_avril', 'Date_avril'),
    'mai': (' cons_mai', 'N_recue_mai', 'Date_mai'),
    'juin': (' cons_juin', 'N_recue_juin', 'Date_juin')
}

# Parcourir df2 pour remplir la table Pay_consommation
for index, row in df2.iterrows():
    n_contrat = row['N_contrat']
    
    for mois, (col_cons, col_recue, col_date) in mois_mapping.items():
        # Récupérer les valeurs de df2
        mnt_paye = row[col_cons]
        n_recue = row[col_recue]
        date_payement = row[col_date]
        
        # Nettoyer et valider les données
        try:
            mnt_paye = float(str(mnt_paye).replace(',', '').strip()) if pd.notna(mnt_paye) else None
            date_paye = f"2024-{list(mois_mapping.keys()).index(mois)+3:02d}-01"  # Format de la date du mois
            date_payement = str(date_payement) if pd.notna(date_payement) else None
            n_recue = str(n_recue) if pd.notna(n_recue) else None
        except ValueError:
            continue  # Si une conversion échoue, on ignore cette ligne
        
        # Vérifier que toutes les valeurs nécessaires sont présentes
        if mnt_paye is not None and n_recue and date_payement:
            # Insérer les données dans la table Pay_consommation
            cursor.execute('''
            INSERT INTO Pay_consommation (N_contrat, N_recue, Mnt_paye, Date_paye, Date_payement, Gestion, Perte)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (n_contrat, n_recue, mnt_paye, date_paye, date_payement, 10, 0))

# Valider les changements et fermer la connexion
conn.commit()
conn.close()

print("La table Pay_consommation a été remplie avec succès.")


La table Pay_consommation a été remplie avec succès.


In [149]:
import pandas as pd
import sqlite3

# Connexion à la base de données SQLite
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Dictionnaire pour mapper les mois aux colonnes correspondantes dans df2
mois_mapping = {
    'mars': (' cons_mars', 'N_recue_mars', 'Date_mars'),
    'avril': (' cons_avril', 'N_recue_avril', 'Date_avril'),
    'mai': (' cons_mai', 'N_recue_mai', 'Date_mai'),
    'juin': (' cons_juin', 'N_recue_juin', 'Date_juin')
}

# Parcourir df2 pour remplir la table Pay_consommation
for index, row in df2.iterrows():
    n_contrat = row['N_contrat']
    
    for mois, (col_cons, col_recue, col_date) in mois_mapping.items():
        # Récupérer les valeurs de df2
        mnt_paye = row[col_cons]
        n_recue = row[col_recue]
        date_payement = row[col_date]
        
        # Nettoyer et valider les données
        try:
            mnt_paye = float(str(mnt_paye).replace(',', '').strip()) if pd.notna(mnt_paye) else None
            date_paye = f"2024-{list(mois_mapping.keys()).index(mois)+3:02d}-01"  # Format de la date du mois
            # Convertir la date de paiement en format 'YYYY-MM-DD' uniquement
            if pd.notna(date_payement):
                date_payement = pd.to_datetime(date_payement).strftime('%Y-%m-%d')
            else:
                date_payement = None
            n_recue = str(n_recue) if pd.notna(n_recue) else None
        except ValueError:
            continue  # Si une conversion échoue, on ignore cette ligne
        
        # Vérifier que toutes les valeurs nécessaires sont présentes
        if mnt_paye is not None and n_recue and date_payement:
            # Insérer les données dans la table Pay_consommation
            cursor.execute('''
            INSERT INTO Pay_consommation (N_contrat, N_recue, Mnt_paye, Date_paye, Date_payement, Gestion, Perte)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (n_contrat, n_recue, mnt_paye, date_paye, date_payement, 10, 0))

# Valider les changements et fermer la connexion
conn.commit()
conn.close()

print("La table Pay_consommation a été remplie avec succès.")


La table Pay_consommation a été remplie avec succès.


In [ ]:
# probleme dans les credit negatif (116 mois 2024-03 dans le fichier de med)

## Payement abonneent

In [ ]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()
cursor.execute('''
DELETE FROM Abonnement
''')
conn.commit()
conn.close()

In [207]:
import pandas as pd

fichier_excel= "/Users/mac/Desktop/Gestion-Eau-Association-Nasr/Donnee Brute/Historique abonement.xlsx"
df = pd.read_excel(fichier_excel, sheet_name="Feuil3 (ABON DEF)")

In [208]:
df.columns

Index(['Code', 'الالاسم الكامل', 'Unnamed: 2', 'مبلغ الدين', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'الرمز',
       'الأسماء', 'واجب المساهمة\n في الربط', 'المؤدى', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Montant payé Abonnements 04 2024',
       'Montant payé Abonnements au 080524', 'Reliquats- abonne au 080524',
       'Unnamed: 20', 'N.Abont', 'Unnamed: 22', 'NOM', 'Situation',
       'مبلغ الدين.1', 'الأداء', 'Reliquat abonnement', 'Montants a payer 2/3',
       'DATE REGLEMENT', 'RECU N°', 'Montant payé Abonnements',
       'Montant payé Abonnements.1', 'Montant payé Abonnements.2',
       'Montant total payé Abonnements'],
      dtype='object')

In [209]:
df = df[['Code','الالاسم الكامل','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Montant payé Abonnements 04 2024']]

df.rename(columns={
    'Code': 'N_contrat',
    'الالاسم الكامل': 'Nom_complet',
    'Unnamed: 13': 'Date_payement',
    'Unnamed: 14': 'N_recue',
    'Unnamed: 15': 'Mnt_paye',
    'Unnamed: 16': 'N_recue_MB',
    'Montant payé Abonnements 04 2024': 'Mnt_paye_MB'
}, inplace=True)

df = df.iloc[:256]
df = df.dropna(subset=['N_contrat'])

In [210]:
df.head()

,N_contrat,Nom_complet,Date_payement,N_recue,Mnt_paye,N_recue_MB,Mnt_paye_MB
0,1.0,عبد الحق الحمزوي,NaN,NaN,NaN,NaN,NaN
1,2.0,صالح حمدي,NaN,NaN,NaN,NaN,NaN
2,3.0,العربي الزعاري,NaN,NaN,NaN,NaN,NaN
3,4.0,أحمد لعناني,NaN,NaN,NaN,NaN,NaN
4,5.0,عبد الكريم النخالي,NaN,NaN,NaN,NaN,NaN


In [211]:
df1= df[['N_contrat','Nom_complet','Mnt_paye','Date_payement','N_recue']]
df2 = df[['N_contrat','Nom_complet','Mnt_paye_MB','N_recue_MB']]

### Inserer les abonnement encaisser par bureau

In [212]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240 entries, 0 to 255
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_contrat      240 non-null    float64
 1   Nom_complet    233 non-null    object 
 2   Mnt_paye       107 non-null    object 
 3   Date_payement  95 non-null     object 
 4   N_recue        93 non-null     object 
dtypes: float64(1), object(4)
memory usage: 11.2+ KB


In [213]:
df1 = df1.dropna(subset=['Mnt_paye'])

# Conversion de N_contrat en chaîne de caractères
df1['N_contrat'] = df1['N_contrat'].astype(int).astype(str)

# Conversion de Mnt_paye en float64
df1['Mnt_paye'] = pd.to_numeric(df1['Mnt_paye'], errors='coerce').astype('float64')

# Conversion de Date_payement en date
df1['Date_payement'] = pd.to_datetime(df1['Date_payement'], errors='coerce')
#df1['Date_payement'] = df1['Date_payement'].dt.date


In [214]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 5 to 252
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   N_contrat      107 non-null    object        
 1   Nom_complet    101 non-null    object        
 2   Mnt_paye       107 non-null    float64       
 3   Date_payement  95 non-null     datetime64[ns]
 4   N_recue        93 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 5.0+ KB


In [215]:
df1.head(10)

,N_contrat,Nom_complet,Mnt_paye,Date_payement,N_recue
5,6,حسن البرني,200.0,2024-07-09,7
6,6,حسن البرني,100.0,2024-08-15,63
7,6,حسن البرني,100.0,2024-09-25,74
8,7,حسن صالح حمدي,175.0,2024-07-30,118
9,7,حسن صالح حمدي,100.0,2024-08-20,83
10,7,حسن صالح حمدي,75.0,2024-10-31,113
14,11,مصطفي أحمد حمدي,100.0,2024-07-15,74
22,19,عبد الرحمان حمدي,200.0,2024-07-12,48
23,19,عبد الرحمان حمدي,150.0,2024-08-19,72
24,19,عبد الرحمان حمدي,100.0,2024-09-21,61


In [216]:
import sqlite3
import pandas as pd

# Connexion à la base de données SQLite
try:
    conn = sqlite3.connect('Gestion_eau.db')
    cursor = conn.cursor()
    print("Connexion à la base de données réussie.")
except sqlite3.Error as e:
    print(f"Erreur lors de la connexion à la base de données : {e}")
    exit()

# Insérer les données dans la table 'abonnement' avec les conditions
try:
    for index, row in df1.iterrows():
        # Vérifier si N_contrat est null
        if pd.notna(row['N_contrat']):
            # Formater Date_payement
            if pd.notna(row['Date_payement']):
                date_payement = row['Date_payement'].strftime('%Y-%m-%d')
            else:
                date_payement = '2024-03-01'  # Valeur par défaut

            # Insérer la ligne dans la table
            cursor.execute('''
                INSERT INTO abonnement (N_contrat, Date_payement, N_recue, Mnt_paye)
                VALUES (?, ?, ?, ?)
            ''', (row['N_contrat'], date_payement, row['N_recue'], row['Mnt_paye']))
    
    # Valider la transaction
    conn.commit()
    print("Données insérées avec succès dans la table 'abonnement'.")
except sqlite3.Error as e:
    print(f"Erreur lors de l'insertion des données : {e}")
    conn.rollback()  # Annuler les modifications en cas d'erreur
finally:
    # Fermer la connexion
    try:
        conn.close()
        print("Connexion à la base de données fermée.")
    except sqlite3.Error as e:
        print(f"Erreur lors de la fermeture de la connexion : {e}")

Connexion à la base de données réussie.
Données insérées avec succès dans la table 'abonnement'.
Connexion à la base de données fermée.


### Inserer les abonnement encaisser par MB

In [217]:
df2.head()

,N_contrat,Nom_complet,Mnt_paye_MB,N_recue_MB
0,1.0,عبد الحق الحمزوي,NaN,NaN
1,2.0,صالح حمدي,NaN,NaN
2,3.0,العربي الزعاري,NaN,NaN
3,4.0,أحمد لعناني,NaN,NaN
4,5.0,عبد الكريم النخالي,NaN,NaN


In [218]:
df2 = df2.dropna(subset=['Mnt_paye_MB'])
df2.head()

,N_contrat,Nom_complet,Mnt_paye_MB,N_recue_MB
5,6.0,حسن البرني,700.0,MB
8,7.0,حسن صالح حمدي,750.0,MB
22,19.0,عبد الرحمان حمدي,600.0,MB
26,21.0,لكبير أوساني,900.0,MB
29,24.0,أحمد قبلي,500.0,MB


In [219]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 5 to 249
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N_contrat    53 non-null     float64
 1   Nom_complet  53 non-null     object 
 2   Mnt_paye_MB  53 non-null     float64
 3   N_recue_MB   48 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.1+ KB


In [220]:
# Convertir N_contrat en entier, puis en chaîne de caractères
df2['N_contrat'] = df2['N_contrat'].astype(int).astype(str)

# Ajouter les colonnes manquantes avec les valeurs spécifiées
df2['Date_payement'] = '2024-03-01'
df2['N_recue'] = 'MB'

# Renommer les colonnes pour correspondre à la table 'abonnement'
df2 = df2.rename(columns={
    'Mnt_paye_MB': 'Mnt_paye',
    'N_recue_MB': 'N_recue'
})

df2['Date_payement'] = pd.to_datetime(df2['Date_payement'], errors='coerce')

In [221]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 5 to 249
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   N_contrat      53 non-null     object        
 1   Nom_complet    53 non-null     object        
 2   Mnt_paye       53 non-null     float64       
 3   N_recue        48 non-null     object        
 4   Date_payement  53 non-null     datetime64[ns]
 5   N_recue        53 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 2.9+ KB


In [222]:
df2.head()

,N_contrat,Nom_complet,Mnt_paye,N_recue,Date_payement,N_recue
5,6,حسن البرني,700.0,MB,2024-03-01,MB
8,7,حسن صالح حمدي,750.0,MB,2024-03-01,MB
22,19,عبد الرحمان حمدي,600.0,MB,2024-03-01,MB
26,21,لكبير أوساني,900.0,MB,2024-03-01,MB
29,24,أحمد قبلي,500.0,MB,2024-03-01,MB


In [223]:
import sqlite3
import pandas as pd

# Connexion à la base de données SQLite
try:
    conn = sqlite3.connect('Gestion_eau.db')
    cursor = conn.cursor()
    print("Connexion à la base de données réussie.")
except sqlite3.Error as e:
    print(f"Erreur lors de la connexion à la base de données : {e}")
    exit()

# Insérer les données dans la table 'abonnement'
try:
    for index, row in df2.iterrows():
        # Vérifier si N_contrat n'est pas null
        if pd.notna(row['N_contrat']):
            # Formater Date_payement
            if pd.notna(row['Date_payement']):
                date_payement = row['Date_payement'].strftime('%Y-%m-%d')
            else:
                date_payement = '2024-03-01'  # Valeur par défaut

            # Insérer la ligne dans la table
            cursor.execute('''
                INSERT INTO abonnement (N_contrat, Date_payement, N_recue, Mnt_paye)
                VALUES (?, ?, ?, ?)
            ''', (row['N_contrat'], date_payement, 000, row['Mnt_paye']))
    
    # Valider la transaction
    conn.commit()
    print("Données insérées avec succès dans la table 'abonnement'.")
except sqlite3.Error as e:
    print(f"Erreur lors de l'insertion des données : {e}")
    conn.rollback()  # Annuler les modifications en cas d'erreur
finally:
    # Fermer la connexion
    try:
        conn.close()
        print("Connexion à la base de données fermée.")
    except sqlite3.Error as e:
        print(f"Erreur lors de la fermeture de la connexion : {e}")

Connexion à la base de données réussie.
Données insérées avec succès dans la table 'abonnement'.
Connexion à la base de données fermée.


### Inserer les abonnement encaiser par bureau precedent

In [247]:
import pandas as pd
fichier_excel= "/Users/mac/Desktop/Gestion-Eau-Association-Nasr/Donnee Brute/MB suivi de compte.xlsx"
df = pd.read_excel(fichier_excel, sheet_name="Sheet4")

In [248]:
df3 = df

In [249]:
df3.head()

,N_contrat,Nom_complet,DUE,PAYE
0,1.0,عبد الحق الحمزوي,3600.0,3600.0
1,2.0,صالح حمدي,3600.0,3600.0
2,3.0,العربي الزعاري,3600.0,3600.0
3,5.0,عبد الكريم النخالي,3600.0,3600.0
4,6.0,حسن البرني,3600.0,2500.0


In [250]:
df3 = df3.dropna(subset=['N_contrat'])
df3['N_contrat'] = df3['N_contrat'].astype(int).astype(str)  # Convertir N_contrat en chaîne de caractères
df3['PAYE'] = df3['PAYE'].astype(float)  # Convertir PAYE en float
df3['Date_payement'] = '2023-12-31'  # Ajouter une colonne Date_payement avec la valeur '2023-12-31'
df3['Date_payement'] = pd.to_datetime(df3['Date_payement'])

/var/folders/jn/9ky4bnns2n72gwm_jg0ksgk80000gn/T/ipykernel_2177/3941810325.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['N_contrat'] = df3['N_contrat'].astype(int).astype(str)  # Convertir N_contrat en chaîne de caractères
/var/folders/jn/9ky4bnns2n72gwm_jg0ksgk80000gn/T/ipykernel_2177/3941810325.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['PAYE'] = df3['PAYE'].astype(float)  # Convertir PAYE en float
/var/folders/jn/9ky4bnns2n72gwm_jg0ksgk80000gn/T/ipykernel_2177/3941810325.py:4: Set

In [251]:
display(df3.info())
display(df3.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 0 to 156
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   N_contrat      152 non-null    object        
 1   Nom_complet    152 non-null    object        
 2   DUE            152 non-null    float64       
 3   PAYE           152 non-null    float64       
 4   Date_payement  152 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 7.1+ KB


None

N_contrat        0
Nom_complet      0
DUE              0
PAYE             0
Date_payement    0
dtype: int64

In [253]:
import sqlite3

# Connexion à la base de données SQLite
try:
    conn = sqlite3.connect('Gestion_eau.db')
    cursor = conn.cursor()
    print("Connexion à la base de données réussie.")
except sqlite3.Error as e:
    print(f"Erreur lors de la connexion à la base de données : {e}")
    exit()

# Insérer les données dans la table 'abonnement'
try:
    for index, row in df3.iterrows():
        # Vérifier si N_contrat n'est pas null
        if pd.notna(row['N_contrat']):
            # Formater Date_payement pour ne garder que la partie date (sans l'heure)
            if pd.notna(row['Date_payement']):
                date_payement = row['Date_payement'].strftime('%Y-%m-%d')  # Format AAAA-MM-JJ
            else:
                date_payement = '2023-12-31'  # Valeur par défaut si Date_payement est null

            # Insérer la ligne dans la table
            cursor.execute('''
                INSERT INTO abonnement (N_contrat, Date_payement, N_recue, Mnt_paye)
                VALUES (?, ?, ?, ?)
            ''', (row['N_contrat'], date_payement, 000, row['PAYE']))
    
    # Valider la transaction
    conn.commit()
    print("Données insérées avec succès dans la table 'abonnement'.")
except sqlite3.Error as e:
    print(f"Erreur lors de l'insertion des données : {e}")
    conn.rollback()  # Annuler les modifications en cas d'erreur
finally:
    # Fermer la connexion
    try:
        conn.close()
        print("Connexion à la base de données fermée.")
    except sqlite3.Error as e:
        print(f"Erreur lors de la fermeture de la connexion : {e}")

Connexion à la base de données réussie.
Données insérées avec succès dans la table 'abonnement'.
Connexion à la base de données fermée.
